# vector 자료구조

In [1]:
from array import array
import math

class Vector2d :
    typecode ='d'
    
    # '__' 는 사용자가 접근하게 막는 보안장치가 아니라 안전장치이다.
    # 이 변수는 __dict__ 에 저장되어 접근 가능하다.
    # 변수가 변하게 하고 싶지 않다면 대신 '_' 를 prefix로 사용하자. 이를 변경하지 않은 것은 파이썬 개발자 사이의 암묵적 약속이다.
    def __init__(self, x,y) :
        self.__x = float(x)
        self.__y = float(y)
        
    # x,y를 일기 전용 속성으로 만든다.
    @property
    def x(self) :
        return self.__x
    @property
    def y(self) :
        return self.__y
    
    # __iter__ method 를 하면 Vector2d를 반복할 수 있고 이로써 언패킹이 가능하다.
    def __iter__(self) :
        return (i for i in (self.x, self.y))
    
    def __repr__(self) :
        class_name = self.__class__.__name__
        return f"{class_name}({self.x}, {self.y})"
    
    def __str__(self) :
        return str(tuple(self))
    
    def __bytes__(self) :
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))
    
    def __eq__(self, other) :
        return tuple(self) == tuple(other)
    
    # 유클리드 거리
    def __abs__(self) :
        return math.hypot(self.x, self.y)
    
    # abs가 0이면 False 나머지는 True
    def __bool__(self) :
        return bool(abs(self))
    
    def __complex__(self) :
        return complex(self.x, self.y)
    
    #classmethod는 첫번째 함수로 자신의 클래스를 받는다.
    @classmethod
    def frombytes(cls, octets) :
        typecode = chr(octets[0])
        # memoryview 는 구조체를 복사하지 않고 메모리를 공유할 수 있게 해준다.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec='') :
        if fmt_spec.endswith('p') :
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else :
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def __hash__(self) :
        # 클래스에 요소가 2개 이상이라면 각 요소의 해쉬의 배타곱이 권장된다.
        return hash(self.x) ^ hash(self.y)

In [2]:
v1 = Vector2d(3,4)
print(v1.x, v1.y)

3.0 4.0


In [3]:
x,y = v1
print(x,y)

3.0 4.0


In [4]:
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [5]:
print(v1)

(3.0, 4.0)


In [6]:
octets = bytes(v1)
print(octets)
v1.frombytes(octets)

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'


Vector2d(3.0, 4.0)

In [7]:
abs(v1)

5.0

In [8]:
bool(v1), bool(Vector2d(0,0))

(True, False)

In [9]:
print(f"{v1:0.2f}")

(3.00, 4.00)


In [10]:
print(f"{v1:p}")

<5.0, 0.9272952180016122>


In [11]:
print(f"{v1:0.2fp}")

<5.00, 0.93>


In [12]:
v1 = Vector2d(3,4)
v2 = Vector2d(3.1,4.2)
hash(v1), hash(v2)
set([v1,v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

In [13]:
complex(v1)

(3+4j)

In [14]:
# '__' 를 사용한 name mangling 은 안전장치이지 보안장치가 아니다. 
# __x 에 접근하기.
print(v1.__dict__)
print(v1._Vector2d__x)
v1._Vector2d__x += 1
print(v1._Vector2d__x)

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}
3.0
4.0


## \_\_slots\_\_
### 객체는 \_\_dict\_\_ 형태로 딕셔너리에 저장된다. 이는 빠른 접근을 용이하게 하지만 해쉬테이블을 유지하므로 메모리 사용이 크다.
### 따라서 \_\_slots\_\_ 을 사용하여 데이터를 튜플 형태로 저장하여 메모리 사용량을 줄일 수 있다.
<br>

### 단 객체가 가지고 있는 속성 이외의 속성을 추가할 수 없게 된다.
### \_\_weakref\_\_ 를 \_\_slots\_\_ 에 추가하지 않으면 객체가 약한 참조의 대상이 될 수 없다.
### 대규모 데이터를 다루어 메모리가 중요하다면 Numpy 나 Pandas 를 사용하자.

In [15]:
# 위의 객체에서 __solots__ 만 추가시켰다.
from array import array
import math

class Vector2d :
    typecode ='d'
    
    __slots__ = ('__x', '__y')
    
    def __init__(self, x,y) :
        self.__x = float(x)
        self.__y = float(y)
        
    # x,y를 일기 전용 속성으로 만든다.
    @property
    def x(self) :
        return self.__x
    @property
    def y(self) :
        return self.__y
    
    # __iter__ method 를 하면 Vector2d를 반복할 수 있고 이로써 언패킹이 가능하다.
    def __iter__(self) :
        return (i for i in (self.x, self.y))
    
    def __repr__(self) :
        class_name = self.__class__.__name__
        return f"{class_name}({self.x}, {self.y})"
    
    def __str__(self) :
        return str(tuple(self))
    
    def __bytes__(self) :
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))
    
    def __eq__(self, other) :
        return tuple(self) == tuple(other)
    
    # 유클리드 거리
    def __abs__(self) :
        return math.hypot(self.x, self.y)
    
    # abs가 0이면 False 나머지는 True
    def __bool__(self) :
        return bool(abs(self))
    
    def __complex__(self) :
        return complex(self.x, self.y)
    
    #classmethod는 첫번째 함수로 자신의 클래스를 받는다.
    @classmethod
    def frombytes(cls, octets) :
        typecode = chr(octets[0])
        # memoryview 는 구조체를 복사하지 않고 메모리를 공유할 수 있게 해준다.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec='') :
        if fmt_spec.endswith('p') :
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else :
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def __hash__(self) :
        # 클래스에 요소가 2개 이상이라면 각 요소의 해쉬의 배타곱이 권장된다.
        return hash(self.x) ^ hash(self.y)